## Logistic Regression using Adam Optimizer

In [ ]:
# Logistic regression using Adam Optimizer
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf

tf.enable_eager_execution()
tfe = tf.contrib.eager
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# import utils
# Define paramaters for the model
learning_rate = 0.1
batch_size = 64
n_epochs = 25
n_train = 60000
n_test = 10000

from tensorflow.examples.tutorials.mnist import input_data
# Step 1: Read in data
fmnist_folder = 'data/fashion'
#Create dataset load function [Refer fashion mnist github page for util function]
#Create train,validation,test split
data = input_data.read_data_sets(fmnist_folder, one_hot=True)
# train, val, test = utils.read_fmnist(fmnist_folder, flatten=True)

# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices((data.train.images, data.train.labels)).map(lambda x, y: (x, tf.cast(y, tf.float32))).shuffle(buffer_size=1000).batch(batch_size)
dataset_iter = tfe.Iterator(train_data)

# create testing Dataset and batch it
test_data = tf.data.Dataset.from_tensor_slices((data.test.images, data.test.labels)).map(lambda x, y: (x, tf.cast(y, tf.float32))).shuffle(buffer_size=1000).batch(batch_size)


# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
#img, label = iterator.get_next()
train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for test_data

# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y

w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01))
b = tf.Variable(tf.zeros([10]))


# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer

def model_output(img):
  model_output = tf.matmul(img, w) + b
  return model_output

# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function

def loss(img, label):
  logits = model_output(img)
  cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label)
  return tf.reduce_mean(cross_entropy_loss)

# Step 6: define optimizer
# using Adam Optimizer with pre-defined learning rate to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
grad = tfe.implicit_gradients(loss)


# Step 7: calculate accuracy with test set
# preds = tf.nn.softmax(logits)
# correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
# accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

# Step 8: train the model for n_epochs times

total_loss=0.0
batch_accuracy=0.0
for i in range(n_epochs):
    for step, (image_batch, label_batch) in enumerate(tfe.Iterator(train_data)):
        total_loss = loss(image_batch, label_batch)
        prediction = tf.nn.softmax(model_output(image_batch))
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(label_batch, 1))
        batch_accuracy += tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        if (step%1000 == 0):
            print("epoch: {} batch_loss: {} batch_accuracy: {}".format(i, total_loss.numpy(),batch_accuracy.numpy()))
            total_loss=0.
            batch_accuracy=0.
        optimizer.apply_gradients(grad(image_batch, label_batch))

#Step 9: Get the Final test accuracy
prediction = tf.nn.softmax(model_output(data.test.images))
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(data.test.labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print("Testset Accuracy: {:.4f}".format(accuracy))

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
epoch: 0 batch_loss: 2.3034355640411377 batch_accuracy: 0.140625
epoch: 1 batch_loss: 2.7775068283081055 batch_accuracy: 743.8125
epoch: 2 batch_loss: 3.2382421493530273 batch_accuracy: 759.390625
epoch: 3 batch_loss: 2.5180447101593018 batch_accuracy: 762.2135620117188
epoch: 4 batch_loss: 2.469020128250122 batch_accuracy: 764.9375
epoch: 5 batch_loss: 2.536775588989258 batch_accuracy: 767.515625
epoch: 6 batch_loss: 3.484461545944214 batch_accuracy: 768.359375
epoch: 7 batch_loss: 3.2625207901000977 batch_accuracy: 767.9426879882812
epoch: 8 batch_loss: 3.1447901725769043 batch_accuracy: 768.40625
epoch: 9 batch_loss: 3.3887946605682373 batch_accuracy: 769.078125
epoch: 10 batch_loss: 1.9695372581481934 batch_accuracy: 770.015625
epoch: 11 batch_loss: 3.0409328937530518 batch_accuracy:

## Logistic Regression using Gradient Descent Optimizer

In [ ]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf

tf.enable_eager_execution()
tfe = tf.contrib.eager
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# import utils
# Define paramaters for the model
learning_rate = 0.1
batch_size = 64
n_epochs = 25
n_train = 60000
n_test = 10000

from tensorflow.examples.tutorials.mnist import input_data
# Step 1: Read in data
fmnist_folder = 'data/fashion'
#Create dataset load function [Refer fashion mnist github page for util function]
#Create train,validation,test split
data = input_data.read_data_sets(fmnist_folder, one_hot=True)
# train, val, test = utils.read_fmnist(fmnist_folder, flatten=True)

# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices((data.train.images, data.train.labels)).map(lambda x, y: (x, tf.cast(y, tf.float32))).shuffle(buffer_size=1000).batch(batch_size)
dataset_iter = tfe.Iterator(train_data)

# create testing Dataset and batch it
test_data = tf.data.Dataset.from_tensor_slices((data.test.images, data.test.labels)).map(lambda x, y: (x, tf.cast(y, tf.float32))).shuffle(buffer_size=1000).batch(batch_size)


# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
#img, label = iterator.get_next()
train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for test_data

# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y

w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01))
b = tf.Variable(tf.zeros([10]))


# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer

def model_output(img):
  model_output = tf.matmul(img, w) + b
  return model_output

# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function

def loss(img, label):
  logits = model_output(img)
  cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label)
  return tf.reduce_mean(cross_entropy_loss)

# Step 6: define optimizer
# using Adam Optimizer with pre-defined learning rate to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
grad = tfe.implicit_gradients(loss)


# Step 7: calculate accuracy with test set
# preds = tf.nn.softmax(logits)
# correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
# accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

# Step 8: train the model for n_epochs times

total_loss=0.0
batch_accuracy=0.0
for i in range(n_epochs):
    for step, (image_batch, label_batch) in enumerate(tfe.Iterator(train_data)):
        total_loss = loss(image_batch, label_batch)
        prediction = tf.nn.softmax(model_output(image_batch))
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(label_batch, 1))
        batch_accuracy += tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        if (step%1000 == 0):
            print("epoch: {} batch_loss: {} batch_accuracy: {}".format(i, total_loss.numpy(),batch_accuracy.numpy()))
            total_loss=0.
            batch_accuracy=0.
        optimizer.apply_gradients(grad(image_batch, label_batch))

#Step 9: Get the Final test accuracy
prediction = tf.nn.softmax(model_output(data.test.images))
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(data.test.labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print("Testset Accuracy: {:.4f}".format(accuracy))

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
epoch: 0 batch_loss: 2.3143441677093506 batch_accuracy: 0.0625
epoch: 1 batch_loss: 0.2831118404865265 batch_accuracy: 753.4583129882812
epoch: 2 batch_loss: 0.23733747005462646 batch_accuracy: 778.0208129882812
epoch: 3 batch_loss: 0.21846097707748413 batch_accuracy: 784.0676879882812
epoch: 4 batch_loss: 0.20782425999641418 batch_accuracy: 787.1614379882812
epoch: 5 batch_loss: 0.20083370804786682 batch_accuracy: 789.0989379882812
epoch: 6 batch_loss: 0.1957944631576538 batch_accuracy: 790.1926879882812
epoch: 7 batch_loss: 0.19193628430366516 batch_accuracy: 791.3176879882812
epoch: 8 batch_loss: 0.18885937333106995 batch_accuracy: 792.0208129882812
epoch: 9 batch_loss: 0.18633465468883514 batch_accuracy: 792.7864379882812
epoch: 10 batch_loss: 0.18422015011310577 batch_accuracy: 793.